<a href="https://colab.research.google.com/github/InesZhiyiZHANG/NF16/blob/main/%E2%80%9Cfintune_by_zzy_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U huggingface_hub

from huggingface_hub import login

# 登录（只需第一次输入一次 token）
login()


In [2]:
!pip install -U unsloth bitsandbytes datasets trl huggingface_hub

      Successfully uninstalled nvidia-cublas-cu12-12.5.3.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: nvidia-cusparse-cu12
    Found existing installation: nvidia-cusparse-cu12 12.5.1.3
    Uninstalling nvidia-cusparse-cu12-12.5.1.3:
      Successfully uninstalled nvidia-cusparse-cu12-12.5.1.3
  Attempting uninstall: nvidia-cudnn-cu12
    Found existing installation: nvidia-cudnn-cu12 9.3.0.75
    Uninstalling nvidia-cudnn-cu12-9.3.0.75:
      Successfully uninstalled nvidia-cudnn-cu12-9.3.0.75
  Attempting uninstall: nvidia-cusolver-cu12
    Found existing installation: nvidia-cusolver-cu12 11.6.3.83
    Uninstalling nvidia-cusolver-cu12-11.6.3.83:
      Successfully uninstalled nvidia-cusolver-cu12-11.6.3.83
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of

In [3]:
def format_example(example):
    input_text = example["input"]
    output_text = example["output"]

    if input_text:
        formatted_text = f"### Input:\n{input_text}\n\n### Output:\n{output_text}"
    else:
        formatted_text = f"### Output:\n{output_text}"
    return {"text": formatted_text}


In [4]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from datasets import load_dataset, Dataset
from trl import SFTTrainer
from transformers import TrainingArguments

import os

os.environ['WANDB_DISABLED'] = 'true' #防止使用日志功能，因为会上传信息

dataset = load_dataset("json", data_files="/content/first_6000.jsonl", split="train")
dataset = dataset.map(format_example)
dataset_train = dataset.remove_columns([col for col in dataset.column_names if col != "text"])

Map:   0%|          | 0/6004 [00:00<?, ? examples/s]

In [5]:
#加载预训练模型
from unsloth import FastLanguageModel
model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = 2048, #最大文本长度
    dtype = torch.float16, #使用FP16精度
    load_in_4bit = True, #使用4-bit精度也用于节省显存
)

==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [7]:
#第六部分：加入 LoRA 微调模块
#W_new = W_pretrained + A @ B。
#A @ B：是你训练的小模块（秩为 r 的两个小矩阵）；
#你只训练 A 和 B，参数量少，训练快，不容易过拟合。
model = FastLanguageModel.get_peft_model(
    model,
    r=16, #rank参数，小矩阵的维度（越大模型表达能力越强）
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    #在哪些模块上应用loRA微调
    #q_proj, k_proj, v_proj, o_proj：自注意力机制（Query、Key、Value、Output）；
    #gate_proj, up_proj, down_proj：前馈网络（FFN）结构中的变换层
    #这些都是决定模型表达能力的部分
    lora_alpha=16, #  对原始模型输出的影响程度，一般等于r最好
    lora_dropout=0, #相当于防止过拟合参数，0.05 ~ 0.1 可用于更大的数据集；
    bias="none", #微调 bias 效果增益较小，一般不开启，而且省成本
    use_gradient_checkpointing="unsloth", #使用 Unsloth 的高效 checkpoint 技术；True 或 "unsloth" 都可以；显著减少内存消耗，适合大模型。
    random_state=42, #固定随机数
    max_seq_length=2048,#指定模型支持的最大文本长度（输入 + 输出 token 总和）。

)


Unsloth 2025.4.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [10]:
#设置训练器

trainer = SFTTrainer(
    model = model,
    tokenizer=tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",#处理字段为text
    max_seq_length = 2048,
    args=TrainingArguments(
        per_device_train_batch_size=2, # GPU 上的 batch的 size 是 2；
        gradient_accumulation_steps=4, #每累积 4 个小 batch 才更新一次模型参数
        #所以batchsize = 8
        warmup_steps=5, #前几部训练用小学习率
        max_steps=50, #表示最多训练 50 步（迭代次数）
        learning_rate=2e-4, #学习率，控制每次权重更新的幅度；
        fp16=not is_bfloat16_supported(), #如果显卡不支持 bfloat16，则使用 fp16；
        bf16=is_bfloat16_supported(),
        logging_steps=1, #每训练一步就打印一次日志；
        output_dir="catmodel_outputs", #checkpoint（训练进度、权重）都会保存到这
        optim="adamw_8bit", #使用了 bitsandbytes 提供的 8bit 优化器；
        seed=42,
        report_to="none",
    )
)

In [11]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,004 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.830100
2,2.751800
3,2.826500
4,2.570500
5,2.655600
6,2.878000
7,2.687200
8,2.633600
9,2.413100
10,2.618300


TrainOutput(global_step=50, training_loss=2.359423875808716, metrics={'train_runtime': 552.6586, 'train_samples_per_second': 0.724, 'train_steps_per_second': 0.09, 'total_flos': 8456536958533632.0, 'train_loss': 2.359423875808716})

In [21]:
def generate_response(user_input):
    # 构造prompt
    prompt = f"{user_input}"

    # 编码 prompt
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: (v.to("cuda").half() if k != "input_ids" else v.to("cuda")) for k, v in inputs.items()}

    # 调用生成
    outputs = model.generate(
        **inputs,
        max_new_tokens=60,         # ⬅️ 限制生成的token数量，防止太长
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    # 解码
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 抽取response
    if prompt in decoded:
        response = decoded.split(prompt)[-1].strip()
    else:
        response = decoded.strip()

    # 简单处理：如果出现多轮对话（比如新V:），就切掉
    if "\nV:" in response:
        response = response.split("\nV:")[0].strip()

    return response



In [22]:
test_input = "V: Je me sens un peu malade.\nF:"
response = generate_response(test_input)
print(response)


Comment ça va?


In [25]:
from huggingface_hub import login
login()  # 粘贴你的 token
model.push_to_hub("zhiyizhiyi/fintune_model_0426")
tokenizer.push_to_hub("zhiyizhiyi/fintune_model_0426")


No files have been modified since last commit. Skipping to prevent empty commit.


Saved model to https://huggingface.co/zhiyizhiyi/fintune_model_0426


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [12]:
from huggingface_hub import whoami
print(whoami()["name"])


zhiyizhiyi
